### Libraries

In [1]:
import os
tilde = os.path.expanduser('~')

import sys
sys.path.insert(0, tilde + '/Scripts/Fake Folder/Python Libraries')

from jb_libraries import *
%matplotlib inline

### Script settings

In [2]:
close_workbook = 'yes'

csv_path = tilde + '/Scripts/Fake Folder/Accounts and Biz Dev/Recurring/Reseller Discount Percentage Check/CSVs/'
workbook_title = '2019 - 05 - May - Reseller Discount Percent Check'

### Get cost data

In [3]:
msrp = pd.read_sql(
'''
SELECT
p.part_id,
pd.products_name AS part_name,
k.sku_id,
IF(k.sku_outsourced_assembly = 1, pm.consigned_latest_cost, pm.latest_cost) AS latest_cost,
IF(k.sku_outsourced_assembly = 1, pm.consigned_manual_cost, pm.manual_cost) AS manual_cost,
p.products_price
FROM parts p
JOIN skus k ON p.part_id = k.part_id
AND k.sku_status = 'working'
JOIN products_manufacturing pm ON k.sku_id = pm.sku_id
JOIN products_description pd ON p.part_id = pd.part_id
''', db)

col_fix(msrp)
print('%s part_id dupe(s)' % np.sum(msrp['part id'].duplicated()))

55 part_id dupe(s)


In [4]:
pd.read_sql(
'''
SELECT
pm.sku_id,
part_id,
consigned_latest_cost,
latest_cost,
consigned_manual_cost,
manual_cost,
sku_outsourced_assembly
FROM products_manufacturing pm
JOIN skus k ON pm.sku_id = k.sku_id
WHERE pm.sku_id = 13571
''', db)

,sku_id,part_id,consigned_latest_cost,latest_cost,consigned_manual_cost,manual_cost,sku_outsourced_assembly
0,13571,4249,0.0,5.0,0.0,5.0,0


### Get discount data

In [5]:
dis1 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'products_reseller_discount_pricing' AS 'table'
FROM products_reseller_discount_pricing
''', db)

dis2 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'premier_discount_pricing' AS 'table'
FROM premier_discount_pricing
''', db)

dis3 = pd.read_sql(
'''
SELECT
part_id,
discount_price,
'products_discount_quantity' As 'table'
FROM products_discount_quantity
''', db)

dis_main = pd.concat([dis1,dis2,dis3])
dis_main.reset_index(drop = True, inplace = True)

col_fix(dis_main)
dis_main.rename(columns = {'discount price':'percentage off'}, inplace = True)
dis_main['percentage off'] = dis_main['percentage off']/100
dis_main.drop_duplicates(['part id','percentage off'], inplace = True)

### Structure it 

In [6]:
# merge
msrp_check = pd.merge(dis_main, msrp, how = 'right', on = 'part id')

# remove parts for which we do not offer discounts of any kind
msrp_check.dropna(inplace = True)

# get price after discount
msrp_check['products price after discount'] = msrp_check['products price'] * (1 - msrp_check['percentage off'])

# get gross profits
for x in ['latest cost','manual cost']:
    msrp_check[x + ' profit'] = msrp_check['products price after discount'] - msrp_check[x]

# calculate this    
msrp_check['margin'] = msrp_check[['latest cost profit','manual cost profit']].min(axis = 1)/msrp_check['products price']        
msrp_check['margin'] = np.round(msrp_check['margin'], 3)

msrp_check['margin @ discount price'] = msrp_check[['latest cost profit','manual cost profit']].min(axis = 1)/msrp_check['products price after discount']        
msrp_check['margin @ discount price'] = np.round(msrp_check['margin @ discount price'], 3)

# get any line < 0 or any % greater than 30%
old_cols = msrp_check.columns

new_cols = ['part id',
        'sku id',
        'part name',
        'latest cost',
        'manual cost',
        'products price',
        'percentage off',
        'products price after discount',
        'latest cost profit',
        'manual cost profit',
        'margin',
        'margin @ discount price',
        'table']
# check to make sure you got all columns
if len(old_cols) == len(new_cols):
    msrp_check = msrp_check[new_cols]
    pass
else:
    raise ValueError('you left out these columns:')
    print(set(list(old_cols.columns)) - set(new_cols))
    
msrp_check.sort_values('margin', inplace = True)    

### Where discount > 30%

In [7]:
greater = msrp_check[msrp_check['percentage off'] > 0.30]

a = list(set(greater['table']))

ls = []
for i in a:
    if i != 'premier_discount_pricing':
        ls.append(i)
        
if len(ls) == 0:
    print('no applicable table has discounts over 30%')
else:
    display(greater[greater['table'].isin(ls)])

no applicable table has discounts over 30%


### Where there is a profit loss or no profit at some discount %

In [8]:
loss = msrp_check[(msrp_check['margin'] <= 0) | (msrp_check['margin @ discount price'] <= 0)].copy()

url = 'https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/{}/Fees'
loss['PLM page'] = url.format(str(loss['part id'].item()))

if loss.empty:
    print('no losses')
else:
    display(loss)
    print(loss['PLM page'])

,part id,sku id,part name,latest cost,manual cost,products price,percentage off,products price after discount,latest cost profit,manual cost profit,margin,margin @ discount price,table,PLM page
10008,2937,4275,USB 2.0 WiFi Hub with 3 USB Ports,5.35,5.35,9.95,0.6,3.98,-1.37,-1.37,-0.138,-0.344,premier_discount_pricing,https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/2937/Fees


10008    https://volcano.adafruit.com/volcano/ada_plm.php?/Parts/2937/Fees
Name: PLM page, dtype: object


### Write to Excel

In [9]:
if close_workbook == 'yes':

    workbook = xlsxwriter.Workbook(csv_path + workbook_title + '.xlsx',
                                   {'nan_inf_to_errors': True,
                                   'strings_to_numbers': True})

    tabs = ['all data',
            'greater than 30%',
            'margin is zero or negative']

    #=========================
    # Formats
    #=========================

    colors = ['#343635',
              '#2e4874',
              '#7eaba4',
              '#928c85',
              '#347c83',
              '#bfb9d6']
    colors = colors * 2

    title = workbook.add_format({'font_size':25,
                                 'font_name':'Arial (Bold)'})

    subtitle = workbook.add_format({'font_size':15,
                                     'font_name':'Arial (Bold)'})


    col_names = workbook.add_format({'font_name':'Arial (Bold)',
                                     'font_color':'white',
                                     'valign':'vcenter',
                                     'align':'center',
                                     'bg_color':colors[2],
                                     'bottom':1,
                                     'top':1,
                                     'left':1,
                                     'right':1})

    money = workbook.add_format({'num_format':'$#,##0'})
    money2 = workbook.add_format({'num_format':'$#,##0.00'})
    percent = workbook.add_format({'num_format':'0.0%'})
    number = workbook.add_format({'num_format':'#,##0'})
    dummy = workbook.add_format({'font_color':'black'})

    for tab in tabs:
        workbook.add_worksheet(tab) # create each tab

    my_worksheets = {}
    for sht in workbook.worksheets():
        my_worksheets[sht.get_name()] = sht # create dict like tab_name:worksheet_instance

    #=========================
    # Apply same formatting to each worksheet
    #=========================

    for k, v in my_worksheets.items():
        sht = my_worksheets[k]
        sht.write(0, 0,
                  k.title(),
                  title)
        sht.write(1, 0,
                  workbook_title,
                  subtitle)

    #=========================
    # column formats
    #=========================        

    fmt = {'part id':dummy,
           'sku id':dummy,
           'part name':dummy,
           'latest cost':money2,
           'manual cost':money2,
           'products price':money2,
           'percentage off':percent,
           'products price after discount':money2,
           'latest cost profit':money2,
           'manual cost profit':money2,
           'margin':percent,
           'margin @ discount price':percent,
           'table':dummy,
           'PLM page':dummy}   

    #=========================
    # worksheets
    #=========================    

    sheets = list(my_worksheets.keys())
    dfs = [msrp_check,
          greater,
          loss]

    for n in range(len(sheets)):

        sht = my_worksheets[sheets[n]]

        df = dfs[n].copy()

        start_row = 3
        start_col = 0

        for i in range(len(df.columns)):
            sht.write(start_row,
                      start_col + i,
                      df.columns[i].title(),
                      col_names)

            len1 = [len(str(x)) for x in df.iloc[:, i]]
            len2 = [len(df.columns[i])]
            len3 = np.max(len1 + len2)

            sht.set_column(start_col + i,
                           start_col + i,
                           len3 + 5)

            for j in range(len(df)):
                sht.write(start_row + 1 + j, 
                          start_col + i,
                          df.iloc[j,i],
                          fmt[df.columns[i]])

            sht.autofilter(start_row, 
                           start_col,
                           start_row + len(df),
                           start_col + len(df.columns) - 1)

    workbook.close()

In [10]:
print('done')

done
